<a href="https://colab.research.google.com/github/adamstiefel/AI-Business-Agents/blob/main/AI_PowerPoint_One_Pager_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 AI PowerPoint One-Pager Generator 📄✨

## Goal
To take a user-defined topic, have an AI perform web research on it, synthesize the information, and then automatically generate a concise, well-structured, one-page PowerPoint (`.pptx`) summary that can be downloaded.

## How this "Makes Money" (Delivers Business Value)

* 🚀 **Drastically Accelerate Draft Creation:** Saves significant time and effort typically spent on initial research, content outlining, and basic slide formatting for presentations or summaries.
* ⏱️ **Rapid Information Synthesis:** Quickly transforms broad topics or scattered information into a condensed, digestible one-page overview.
* 📊 **Efficient Meeting & Briefing Preparation:** Generate instant pre-reads, executive summaries, or talking point slides for meetings and briefings.
* 💡 **Content Curation & Ideation Kick-starter:** Acts as an intelligent assistant to gather, structure, and present initial information, which can then be refined for reports, articles, or further research.
* 📢 **Quick Proposal & Idea Pitching:** Develop initial one-pagers to present ideas or proposals swiftly.
* 🎓 **Learning & Topic Exploration Aid:** Provides a quick, structured summary for understanding the basics of a new topic.
* 💪 **Focus Human Effort on Refinement:** Allows users to spend their time on higher-value tasks like in-depth analysis, strategic messaging, and visual polishing, rather than starting from scratch.

---
**Important Notes:**
* The "beauty" of the PowerPoint will be focused on a clean, professional, and readable layout generated programmatically. Advanced custom designs would typically require templates or manual editing.
* The quality of the research and content depends on publicly available information and the LLM's capabilities.
* This tool generates a *first draft*. Human review and refinement are essential.
* Directly "viewing" the `.pptx` in Colab is not natively supported; downloading the file is the primary method.
---

In [1]:
#@title 2. Setup and Installations
# --- Install necessary libraries ---
!pip install openai python-pptx requests beautifulsoup4 google-search-results -q

import openai
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor # For setting text color

import requests
from bs4 import BeautifulSoup
import json
import re
from IPython.display import display, Markdown, HTML
from google.colab import files as colab_files # For downloading the pptx
from google.colab import userdata
from serpapi import GoogleSearch

print("Libraries installed/available and imported.")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 6.7 MB/s eta 0:00:00
Libraries installed/available and imported.


In [2]:
#@title 3. API Key Configuration
# --- Configure your OpenAI API Key ---
OPENAI_API_KEY = None
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    if OPENAI_API_KEY:
        openai.api_key = OPENAI_API_KEY
        print("✅ OpenAI API Key loaded successfully from Colab Secrets.")
    else:
        print("⚠️ OpenAI API Key not found in Colab Secrets. Please add it.")
except Exception as e:
    print(f"⚠️ Error accessing Colab Secrets for OpenAI: {e}.")

if not openai.api_key:
    print("🛑 OpenAI API Key is NOT SET. LLM functionality will not work.")

# --- Configure your SerpApi API Key (for Google Search) ---
SERPAPI_API_KEY = None
try:
    SERPAPI_API_KEY = userdata.get('SERPAPI_API_KEY')
    if SERPAPI_API_KEY:
        print("✅ SerpApi API Key loaded successfully from Colab Secrets.")
    else:
        print("⚠️ SerpApi API Key not found in Colab Secrets. Google Search functionality will be limited.")
except Exception as e:
    print(f"⚠️ Error accessing Colab Secrets for SerpApi: {e}.")

if not SERPAPI_API_KEY:
    print("🛑 SerpApi API Key is NOT SET. Google Search using SerpApi will not work effectively.")

✅ OpenAI API Key loaded successfully from Colab Secrets.
✅ SerpApi API Key loaded successfully from Colab Secrets.


In [3]:
#@title 4. User Input: Presentation Topic

#@markdown Enter the topic for your one-page PowerPoint presentation:
presentation_topic = "The Future of Renewable Energy" #@param {type:"string"}

#@markdown (Optional) Specify any key aspects to focus on or particular points to include:
#@markdown (e.g., "Focus on solar and wind, include challenges and opportunities")
specific_focus_points = "Key technologies, economic impact, and environmental benefits" #@param {type:"string"}

#@markdown (Optional) Define a target audience (e.g., "general public", "investors", "students"):
#@markdown This helps the AI tailor the language and complexity.
target_audience = "University students" #@param {type:"string"}


if not presentation_topic or presentation_topic == "e.g., The Impact of AI on Healthcare": # Basic check for default
    print("⚠️ Please enter a valid presentation topic.")
else:
    print(f"✅ Topic: {presentation_topic}")
    if specific_focus_points:
        print(f"   Focus: {specific_focus_points}")
    if target_audience:
        print(f"   Target Audience: {target_audience}")

✅ Topic: The Future of Renewable Energy
   Focus: Key technologies, economic impact, and environmental benefits
   Target Audience: University students


In [4]:
#@title 5. Information Gathering (Web Research)

# --- Helper function to fetch and parse a single URL (basic) ---
def fetch_url_content_for_ppt(url):
    if not url or not url.startswith(('http://', 'https://')):
        return None # Skip invalid URLs
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        for SCRIPT_OR_STYLE in soup(["script", "style", "header", "footer", "nav", "aside", "form", "button"]): SCRIPT_OR_STYLE.decompose()
        text = soup.get_text(separator='\n', strip=True)
        text = re.sub(r'\n\s*\n+', '\n', text)
        return text[:8000] # Limit content from a single page to keep total manageable
    except Exception: # Catch all exceptions for fetching/parsing
        return None

# --- Gather information using Google Search (SerpApi) ---
researched_text_corpus = ""
search_log = "## Research Log:\n\n"

if SERPAPI_API_KEY and presentation_topic and presentation_topic != "e.g., The Impact of AI on Healthcare":
    print(f"🔎 Researching topic: {presentation_topic} using SerpApi...")

    queries = [
        f"{presentation_topic} overview",
        f"key aspects of {presentation_topic}",
        f"introduction to {presentation_topic}",
        f"benefits of {presentation_topic}",
        f"challenges in {presentation_topic}",
        f"future outlook {presentation_topic}",
    ]
    if specific_focus_points:
        queries.append(f"{presentation_topic} focusing on {specific_focus_points}")

    all_fetched_content = []
    urls_processed = set()

    for query_idx, query in enumerate(queries[:5]): # Limit number of search queries
        search_log += f"**Query {query_idx+1}:** {query}\n"
        print(f"  Executing query: {query}")
        try:
            params = {"q": query, "api_key": SERPAPI_API_KEY, "num": 3}
            search = GoogleSearch(params)
            results = search.get_dict()

            organic_results = results.get("organic_results", [])
            if organic_results:
                for res_idx, result in enumerate(organic_results[:2]): # Process top 2 results per query
                    title = result.get("title", "No Title")
                    link = result.get("link")
                    snippet = result.get("snippet", title)
                    search_log += f"  - Result: {title} ({link})\n    Snippet: {snippet}\n"

                    if link and link not in urls_processed:
                        urls_processed.add(link)
                        print(f"    Fetching content from: {link}")
                        content = fetch_url_content_for_ppt(link)
                        if content:
                            all_fetched_content.append(f"Source: {title} ({link})\n\n{content}\n\n---\n\n")
                            search_log += f"    *Fetched content (length: {len(content)} chars)*\n"
                        else:
                            all_fetched_content.append(f"Source: {title} ({link})\n\nSnippet: {snippet}\n\n---\n\n") # Fallback to snippet if fetch fails
                            search_log += f"    *Failed to fetch full content, using snippet.*\n"
                    elif link in urls_processed:
                        search_log += f"    *URL already processed.*\n"
                    else: # No link, just use snippet
                         all_fetched_content.append(f"Source: {title}\n\nSnippet: {snippet}\n\n---\n\n")
            else:
                search_log += "  - No organic results.\n"
        except Exception as e:
            search_log += f"  - Error during search: {str(e)[:200]}\n"
        search_log += "\n"

    researched_text_corpus = "".join(all_fetched_content)

    if not researched_text_corpus.strip():
        researched_text_corpus = "No detailed information could be gathered via automated search. The presentation will be very high-level or based on general knowledge if the LLM proceeds."
        print(f"⚠️ {researched_text_corpus}")
else:
    error_msg = "SerpApi API Key not set or topic not provided. Skipping web research."
    researched_text_corpus = f"Web research skipped: {error_msg}"
    search_log += error_msg
    print(f"🛑 {error_msg}")

print("\n--- Research Log Summary ---")
display(Markdown(search_log[:2000] + "... (log truncated for display)")) # Display a portion of the log
# print("\n--- Full Researched Text Corpus (first 2000 chars for LLM input preview) ---")
# display(Markdown("```\n" + researched_text_corpus[:2000] + "\n```"))

🔎 Researching topic: The Future of Renewable Energy using SerpApi...
  Executing query: The Future of Renewable Energy overview
    Fetching content from: https://earth.org/the-growth-of-renewable-energy-what-does-the-future-hold/
    Fetching content from: https://www.ibm.com/think/insights/renewable-energy-future
  Executing query: key aspects of The Future of Renewable Energy
  Executing query: introduction to The Future of Renewable Energy
  Executing query: benefits of The Future of Renewable Energy
    Fetching content from: https://www.un.org/en/climatechange/raising-ambition/renewable-energy
    Fetching content from: https://www.energy.gov/eere/renewable-energy-pillar
  Executing query: challenges in The Future of Renewable Energy
    Fetching content from: https://iee.psu.edu/news/blog/transitioning-renewable-energy-challenges-and-opportunities
    Fetching content from: https://regenpower.com/what-are-the-problems-faced-by-renewable-energy/

--- Research Log Summary ---


## Research Log:

**Query 1:** The Future of Renewable Energy overview
  - Result: What the Future of Renewable Energy Looks Like (https://earth.org/the-growth-of-renewable-energy-what-does-the-future-hold/)
    Snippet: 1. Solar Will Become 35% Cheaper By 2024 · 2. Onshore Wind Energy Capacity Will Increase 57% By 2024 · 3. Hydroelectric Capacity Will Rise 9% By ...
    *Fetched content (length: 8000 chars)*
  - Result: The Future of Renewable Energy (https://www.ibm.com/think/insights/renewable-energy-future)
    Snippet: The next generation of clean energy needs innovative technology and power generation to help the world reach net-zero emissions.
    *Fetched content (length: 8000 chars)*

**Query 2:** key aspects of The Future of Renewable Energy
  - Result: The Future of Renewable Energy (https://www.ibm.com/think/insights/renewable-energy-future)
    Snippet: The next generation of clean energy needs innovative technology and power generation to help the world reach net-zero emissions.
    *URL already processed.*
  - Result: What the Future of Renewable Energy Looks Like (https://earth.org/the-growth-of-renewable-energy-what-does-the-future-hold/)
    Snippet: 1. Solar Will Become 35% Cheaper By 2024 · 2. Onshore Wind Energy Capacity Will Increase 57% By 2024 · 3. Hydroelectric Capacity Will Rise 9% By ...
    *URL already processed.*

**Query 3:** introduction to The Future of Renewable Energy
  - Result: The Future of Renewable Energy (https://www.ibm.com/think/insights/renewable-energy-future)
    Snippet: The next generation of clean energy needs innovative technology and power generation to help the world reach net-zero emissions.
    *URL already processed.*
  - Result: What the Future of Renewable Energy Looks Like (https://earth.org/the-growth-of-renewable-energy-what-does-the-future-hold/)
    Snippet: According to The International Energy Agency, renewable energy capacity is set to expand by 50% by 2024, led by solar energy.
    *URL already proces... (log truncated for display)

In [5]:
#@title 6. Content Structuring & Synthesis (LLM)

#@markdown Select LLM model for content generation (GPT-4 class recommended):
llm_model_ppt = "gpt-4-turbo-preview" #@param ["gpt-4-turbo-preview", "gpt-4", "gpt-3.5-turbo"]

structured_content_for_ppt = None

def generate_ppt_content_structure(topic, research_text, focus, audience, model_name="gpt-4-turbo-preview"):
    if not openai.api_key:
        return {"error": "OpenAI API Key not set."}
    if not research_text or "Web research skipped" in research_text or "No detailed information could be gathered" in research_text :
        # If research failed, try to generate content based on the topic and focus alone as a fallback
        research_context_for_prompt = "Assume general knowledge for the topic as web research was limited or failed."
        print("⚠️ Web research was limited. LLM will attempt to generate content based on general knowledge of the topic.")
    else:
        research_context_for_prompt = f"Leverage the following research text:\n<research_text>\n{research_text}\n</research_text>"


    prompt = f"""
    You are an expert Research Analyst and Content Creator tasked with drafting the content for a concise, informative one-page PowerPoint presentation.
    The presentation is on the topic: "{topic}".
    Your target audience is: "{audience}".
    Specific focus points to include if possible: "{focus}".

    {research_context_for_prompt}

    Based *primarily* on the provided research text (if available and substantial, otherwise general knowledge), structure the content for a single PowerPoint slide as a JSON object with the following keys:
    - "title": A compelling and concise title for the one-page presentation (max 10-12 words).
    - "introduction": A brief introductory overview of the topic (2-3 sentences, max 50-70 words).
    - "key_sections": A list of 3 to 4 key sections. Each section in the list should be an object with:
        - "header": A short, impactful header for the section (max 5-7 words).
        - "points": A list of 2 to 4 concise bullet points (each bullet point max 15-20 words).
    - "conclusion": A brief concluding remark or key takeaway (1-2 sentences, max 40-50 words).

    Ensure the language is appropriate for the target audience: "{audience}".
    The content must be highly summarized and suitable for a single, readable PowerPoint slide.
    Prioritize clarity, conciseness, and impact. Do not use overly complex sentences.
    Avoid explicitly saying "This presentation will..." or "In this slide...". Just provide the content.

    Output ONLY the JSON object. Do not include any other explanatory text before or after the JSON.
    Example of a key_section object: {{ "header": "Key Benefits", "points": ["Benefit 1 description.", "Benefit 2 description."] }}
    """

    print(f"\n🤖 Synthesizing content for PowerPoint on '{topic}' using {model_name}...")
    print("This may take a moment...")

    try:
        response = openai.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are an expert content creator for presentations, outputting structured JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4, # Balance of factual and well-structured
            max_tokens=1500, # Allow for structured JSON output
            response_format={"type": "json_object"} # Request JSON output if model supports it
        )
        content_json_str = response.choices[0].message.content

        # Clean the JSON string if it's wrapped in markdown code block
        if content_json_str.startswith("```json"):
            content_json_str = content_json_str[len("```json"):]
        if content_json_str.endswith("```"):
            content_json_str = content_json_str[:-len("```")]
        content_json_str = content_json_str.strip()

        parsed_content = json.loads(content_json_str)
        print("✅ Content structure generated by LLM.")
        # Basic validation
        if not all(k in parsed_content for k in ["title", "introduction", "key_sections", "conclusion"]):
            print("⚠️ LLM output was not in the expected JSON structure. Presentation might be incomplete.")
            return {"error": "LLM output structure incorrect.", "raw_output": content_json_str}
        return parsed_content
    except json.JSONDecodeError as e:
        print(f"⚠️ Error decoding JSON from LLM: {e}")
        print(f"LLM Raw Output was:\n{content_json_str}")
        return {"error": f"JSONDecodeError: {e}", "raw_output": content_json_str}
    except Exception as e:
        print(f"⚠️ An error occurred during LLM content generation: {e}")
        return {"error": str(e)}

# --- Main execution ---
if 'presentation_topic' in globals() and presentation_topic and presentation_topic != "e.g., The Impact of AI on Healthcare":
    if openai.api_key:
        structured_content_for_ppt = generate_ppt_content_structure(
            presentation_topic,
            researched_text_corpus,
            specific_focus_points,
            target_audience,
            model_name=llm_model_ppt
        )
        if "error" in structured_content_for_ppt:
            print(f"\n🛑 Failed to generate structured content: {structured_content_for_ppt['error']}")
            if "raw_output" in structured_content_for_ppt:
                display(Markdown("```\n" + structured_content_for_ppt['raw_output'] + "\n```"))
        else:
            print("\n--- Generated Content Structure (for PowerPoint) ---")
            display(Markdown(f"**Title:** {structured_content_for_ppt.get('title', 'N/A')}"))
            display(Markdown(f"**Introduction:** {structured_content_for_ppt.get('introduction', 'N/A')}"))
            for i, section in enumerate(structured_content_for_ppt.get('key_sections', [])):
                display(Markdown(f"\n**Section {i+1}: {section.get('header', 'N/A')}**"))
                for point in section.get('points', []):
                    display(Markdown(f"* {point}"))
            display(Markdown(f"\n**Conclusion:** {structured_content_for_ppt.get('conclusion', 'N/A')}"))
    else:
        print("\n🛑 OpenAI API Key not configured. Cannot synthesize content.")
else:
    print("\n⚠️ Please provide a presentation topic in Step 4.")


🤖 Synthesizing content for PowerPoint on 'The Future of Renewable Energy' using gpt-4-turbo-preview...
This may take a moment...
✅ Content structure generated by LLM.

--- Generated Content Structure (for PowerPoint) ---


**Title:** Renewable Energy: Pathway to a Sustainable Future

**Introduction:** Explore the dynamic landscape of renewable energy, its rapid growth, and the pivotal role it plays in shaping a sustainable, eco-friendly future.


**Section 1: Rapid Expansion of Solar Power**

* Solar capacity set to double, becoming cheaper by 35% by 2024.

...


**Section 3: Technological Innovations and Challenges**

* Next-gen solar panels and wind turbines enhance efficiency.

* Energy storage solutions critical for managing supply and demand.

* Infrastructure and policy adaptation remain key challenges.


**Conclusion:** Renewable energy stands at the forefront of combating climate change, promising a greener, more sustainable future with economic growth and technological innovation.

In [6]:
#@title 7. PowerPoint Generation

#@markdown Define output filename:
output_filename = "AI_Generated_OnePager.pptx" #@param {type:"string"}

def create_one_page_ppt(content_structure, filename="presentation.pptx"):
    if not content_structure or "error" in content_structure:
        print("🛑 Cannot generate PowerPoint: Content structure is missing or invalid.")
        return None

    prs = Presentation()
    # Use a blank slide layout (typically layout index 6 in a new presentation)
    blank_slide_layout = prs.slide_layouts[6]
    slide = prs.slides.add_slide(blank_slide_layout)

    # Slide dimensions (standard 16:9 aspect ratio in inches)
    slide_width = Inches(10) # Standard width for 16:9
    slide_height = Inches(5.625) # Standard height for 16:9
    prs.slide_width = slide_width
    prs.slide_height = slide_height

    # Margins
    left_margin = Inches(0.3)
    right_margin = Inches(0.3)
    top_margin = Inches(0.25)
    bottom_margin = Inches(0.25)
    content_width = slide_width - left_margin - right_margin

    # --- Title ---
    title_shape = slide.shapes.add_textbox(left_margin, top_margin, content_width, Inches(0.75))
    title_tf = title_shape.text_frame
    title_tf.clear() # Ensure it's empty before adding new paragraph
    p_title = title_tf.paragraphs[0]
    p_title.text = content_structure.get("title", "Presentation Title")
    p_title.font.name = 'Arial'
    p_title.font.size = Pt(32)
    p_title.font.bold = True
    p_title.alignment = PP_ALIGN.CENTER
    p_title.font.color.rgb = RGBColor(0x00, 0x00, 0x5A) # Dark Blue
    current_y = top_margin + Inches(0.75 + 0.15) # Title height + small gap

    # --- Introduction ---
    intro_height = Inches(0.5)
    intro_shape = slide.shapes.add_textbox(left_margin, current_y, content_width, intro_height)
    intro_tf = intro_shape.text_frame
    intro_tf.clear()
    p_intro = intro_tf.paragraphs[0]
    p_intro.text = content_structure.get("introduction", "Introduction text goes here.")
    p_intro.font.name = 'Arial'
    p_intro.font.size = Pt(12)
    intro_tf.word_wrap = True
    current_y += intro_height + Inches(0.15)

    # --- Key Sections (attempting a 2-column layout if 3 or 4 sections) ---
    key_sections = content_structure.get("key_sections", [])
    num_sections = len(key_sections)

    available_height_for_sections = slide_height - current_y - bottom_margin - Inches(0.5) # Reserve space for conclusion

    if num_sections > 0:
        if num_sections >= 2 and num_sections <= 4 : # Try 2 columns
            col_width = (content_width - Inches(0.2)) / 2 # width for each column, with a small gap
            items_per_col = (num_sections + 1) // 2 # roughly half in each

            col1_sections = key_sections[:items_per_col]
            col2_sections = key_sections[items_per_col:]

            max_y_col1 = current_y
            max_y_col2 = current_y

            # Column 1
            temp_y_col1 = current_y
            for section in col1_sections:
                sec_header = section.get("header", "Section Header")
                sec_points = section.get("points", [])

                # Header
                h_shape = slide.shapes.add_textbox(left_margin, temp_y_col1, col_width, Inches(0.3))
                h_tf = h_shape.text_frame
                h_tf.clear()
                p_h = h_tf.paragraphs[0]
                p_h.text = sec_header
                p_h.font.name = 'Arial'
                p_h.font.size = Pt(16)
                p_h.font.bold = True
                p_h.font.color.rgb = RGBColor(0x1E, 0x4E, 0x8C) # Another shade of blue
                temp_y_col1 += Inches(0.3 + 0.05)

                # Points
                for point_text in sec_points:
                    pt_shape = slide.shapes.add_textbox(left_margin + Inches(0.2), temp_y_col1, col_width - Inches(0.2), Inches(0.25 * (point_text.count('\n') + 1))) # Dynamic height approx
                    pt_tf = pt_shape.text_frame
                    pt_tf.clear()
                    p_pt = pt_tf.paragraphs[0]
                    p_pt.text = f"• {point_text}" # Add bullet character
                    p_pt.font.name = 'Arial'
                    p_pt.font.size = Pt(11)
                    pt_tf.word_wrap = True
                    # Estimate height based on text length; this is tricky
                    est_height = Inches(0.20) + Inches(0.15 * (len(point_text) // 40)) # Rough estimate
                    temp_y_col1 += est_height # Increment y for next point
                temp_y_col1 += Inches(0.15) # Gap after section
            max_y_col1 = temp_y_col1

            # Column 2
            temp_y_col2 = current_y
            col2_left = left_margin + col_width + Inches(0.2)
            for section in col2_sections:
                sec_header = section.get("header", "Section Header")
                sec_points = section.get("points", [])
                # Header
                h_shape = slide.shapes.add_textbox(col2_left, temp_y_col2, col_width, Inches(0.3))
                h_tf = h_shape.text_frame; h_tf.clear(); p_h = h_tf.paragraphs[0]
                p_h.text = sec_header; p_h.font.name = 'Arial'; p_h.font.size = Pt(16); p_h.font.bold = True; p_h.font.color.rgb = RGBColor(0x1E, 0x4E, 0x8C)
                temp_y_col2 += Inches(0.3 + 0.05)
                # Points
                for point_text in sec_points:
                    pt_shape = slide.shapes.add_textbox(col2_left + Inches(0.2), temp_y_col2, col_width - Inches(0.2), Inches(0.25 * (point_text.count('\n') + 1)))
                    pt_tf = pt_shape.text_frame; pt_tf.clear(); p_pt = pt_tf.paragraphs[0]
                    p_pt.text = f"• {point_text}"; p_pt.font.name = 'Arial'; p_pt.font.size = Pt(11); pt_tf.word_wrap = True
                    est_height = Inches(0.20) + Inches(0.15 * (len(point_text) // 40))
                    temp_y_col2 += est_height
                temp_y_col2 += Inches(0.15)
            max_y_col2 = temp_y_col2
            current_y = max(max_y_col1, max_y_col2) # New Y is below the longest column

        else: # Single column for 1 section or > 4 sections (basic layout)
            for section in key_sections:
                sec_header = section.get("header", "Section Header")
                sec_points = section.get("points", [])
                # Header
                h_shape = slide.shapes.add_textbox(left_margin, current_y, content_width, Inches(0.3))
                h_tf = h_shape.text_frame; h_tf.clear(); p_h = h_tf.paragraphs[0]
                p_h.text = sec_header; p_h.font.name = 'Arial'; p_h.font.size = Pt(16); p_h.font.bold = True; p_h.font.color.rgb = RGBColor(0x1E, 0x4E, 0x8C)
                current_y += Inches(0.3 + 0.05)
                # Points
                for point_text in sec_points:
                    pt_shape = slide.shapes.add_textbox(left_margin + Inches(0.2), current_y, content_width - Inches(0.2), Inches(0.25 * (point_text.count('\n') + 1)))
                    pt_tf = pt_shape.text_frame; pt_tf.clear(); p_pt = pt_tf.paragraphs[0]
                    p_pt.text = f"• {point_text}"; p_pt.font.name = 'Arial'; p_pt.font.size = Pt(11); pt_tf.word_wrap = True
                    est_height = Inches(0.20) + Inches(0.15 * (len(point_text) // 40))
                    current_y += est_height
                current_y += Inches(0.15)

    # --- Conclusion ---
    # Place conclusion at a fixed position from bottom or after sections
    conclusion_y = slide_height - bottom_margin - Inches(0.5) # Fixed from bottom
    if current_y > conclusion_y - Inches(0.2): # If content is too long, adjust conclusion y
        conclusion_y = current_y + Inches(0.1)
        if conclusion_y > slide_height - bottom_margin - Inches(0.25): # Cap height
             conclusion_y = slide_height - bottom_margin - Inches(0.25)


    conc_shape = slide.shapes.add_textbox(left_margin, conclusion_y, content_width, Inches(0.5))
    conc_tf = conc_shape.text_frame
    conc_tf.clear()
    p_conc = conc_tf.paragraphs[0]
    p_conc.text = content_structure.get("conclusion", "Conclusion text goes here.")
    p_conc.font.name = 'Arial'
    p_conc.font.size = Pt(12)
    p_conc.font.italic = True
    conc_tf.word_wrap = True
    p_conc.alignment = PP_ALIGN.CENTER

    try:
        prs.save(filename)
        print(f"✅ PowerPoint presentation '{filename}' saved successfully.")
        return filename
    except Exception as e:
        print(f"🛑 Error saving PowerPoint: {e}")
        return None

# --- Main execution ---
if 'structured_content_for_ppt' in globals() and \
   structured_content_for_ppt and \
   not structured_content_for_ppt.get("error"):

    generated_ppt_file = create_one_page_ppt(structured_content_for_ppt, output_filename)

    if generated_ppt_file:
        print(f"\n⬇️ To download '{generated_ppt_file}', run the next cell or find it in the Colab file explorer.")
        # Offer download directly
        try:
          colab_files.download(generated_ppt_file)
          print(f"   Download for '{generated_ppt_file}' initiated.")
        except Exception as e:
          print(f"   Automatic download failed: {e}. Please use file explorer to download.")

elif 'structured_content_for_ppt' in globals() and structured_content_for_ppt and structured_content_for_ppt.get("error"):
    print("\n⚠️ Skipping PowerPoint generation due to errors in content synthesis (Step 6).")
else:
    print("\n⚠️ Content for PowerPoint not generated (Step 6 was likely not run or failed). Skipping PowerPoint generation.")

✅ PowerPoint presentation 'AI_Generated_OnePager.pptx' saved successfully.

⬇️ To download 'AI_Generated_OnePager.pptx', run the next cell or find it in the Colab file explorer.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   Download for 'AI_Generated_OnePager.pptx' initiated.
